In [62]:
# ------------------------- Incidents -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Chargement des 2 dataset
df1 = pd.read_csv(
    "../data/LFB Incident data from 2009 - 2017.csv", sep=",", low_memory=False
)
df2 = pd.read_csv(
    "../data/LFB Incident data from 2018 onwards.csv", sep=",", low_memory=False
)

In [ ]:
# concaténation
df = pd.concat([df1, df2], axis=0)
# reset index
df = df.reset_index(drop=True)

# suppression des ".00" sur certaines lignes de "IncidentNumber"
df["IncidentNumber"] = df["IncidentNumber"].apply(
    lambda x: str(x)[0:-3] if (".00" in x) & (str(x)[-3:] == ".00") else x
)

# formatage des dates
df.DateOfCall = pd.to_datetime(df.DateOfCall, format="%d-%b-%y")
df.TimeOfCall = pd.to_datetime(df.TimeOfCall, format="%H:%M:%S")

# création de champs Mois et DayOfWeek
df["Month"] = df["DateOfCall"].dt.month
df["DayOfWeek"] = df["DateOfCall"].dt.dayofweek + 1

display(df.head())
display(df.info())

In [ ]:
def TestDoublonSousCatégorie(categorie, ssCategorie):
    # Vérifie si les StopCodeDescription existe en double dans les IncidentGroup
    result = df.groupby(ssCategorie)[categorie].nunique()
    print(result[result.values > 1])
    print("Na pour", categorie, df[categorie].isna().sum())
    print("Na pour", ssCategorie, df[ssCategorie].isna().sum())

TestDoublonSousCatégorie("IncidentGroup", "StopCodeDescription")
print("\n", "6 na sur IncidentGroup et pas de doublon, peut supprimer la colonne IncidentGroup")

print()
TestDoublonSousCatégorie("PropertyCategory", "PropertyType")
print("\n", "6 na sur PropertyCategory et 6 pour PropertyType, peut supprimer la colonne PropertyCategory")

In [ ]:
# Merge de SpecialService dans Stopcode
StopCodeDescription	

SpecialServiceType

drop SpecialServiceType 





In [65]:
# suppression des données inutiles, en doublon métier, ou non présente au moment de la prédiction
df = df.drop(
    [
        "DateOfCall",
        "TimeOfCall",
        "IncidentGroup",
        "PropertyCategory",
        "Postcode_full",
        "UPRN",
        "IncGeo_BoroughName",
        "ProperCase",
        "IncGeo_WardName",
        "IncGeo_WardNameNew",
        "Easting_m",
        "Northing_m",
        "Easting_rounded",
        "Northing_rounded",
        "Latitude",
        "Longitude",
        "FRS",
        "IncidentStationGround",
        "FirstPumpArriving_AttendanceTime",
        "SecondPumpArriving_AttendanceTime",
        "SecondPumpArriving_DeployedFromStation",
        "NumStationsWithPumpsAttending",
        "PumpCount",
        "PumpMinutesRounded",
        "Notional Cost (£)",
        "NumCalls",
    ],
    axis=1,
)

In [66]:
# sauvegarde d'un fichier temporaire
df.to_csv("../data/PreIncidents.csv", sep=";", index=False) 

In [67]:
# ------------------------- Mobilisations -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Chargement des 3 dataset
df1 = pd.read_csv(
    "../data/LFB Mobilisation data from January 2009 - 2014.csv",
    sep=";",
    low_memory=False,
)
df2 = pd.read_csv(
    "../data/LFB Mobilisation data from 2015 - 2020.csv", sep=";", low_memory=False
)
df3 = pd.read_csv(
    "../data/LFB Mobilisation data from 2021 - 2024.csv",
    sep=",",
    low_memory=False,
    usecols=lambda column: column not in ["BoroughName", "WardName"],
)

In [ ]:
# concaténation
df = pd.concat([df1, df2, df3], axis=0)
# reset index
df = df.reset_index(drop=True)

# formatage des dates
df.DateAndTimeMobilised = pd.to_datetime(df.DateAndTimeMobilised, format='%d/%m/%Y %H:%M')
df.DateAndTimeMobile = pd.to_datetime(df.DateAndTimeMobile, format="%d/%m/%Y %H:%M")
df.DateAndTimeArrived = pd.to_datetime(df.DateAndTimeArrived, format="%d/%m/%Y %H:%M")
df.DateAndTimeLeft = pd.to_datetime(df.DateAndTimeLeft, format="%d/%m/%Y %H:%M")
df.DateAndTimeReturned = pd.to_datetime(df.DateAndTimeReturned, format="%d/%m/%Y %H:%M")

display(df.head())
display(df.info())

In [ ]:
# création de la colonne Temps sur site en seconde de la pompe par delta de temps arrivée et départ
df["PumpOnSiteSeconds"] = (df.DateAndTimeLeft - df.DateAndTimeArrived).dt.seconds
display(df.head(3))

# Regroupe les lignes par incidents, car il y a une ligne par incidents / camion avec un n° d'ordre
# Comme on s'intéresse à prédire des temps, on calcule les temps par incidents, min, max et moyen
# et on cherchera à prédire des tgemps
# les infos de 1ere station sur place et nb de pompe seront prises à partir de incidents
aggregated = (
    df.groupby("IncidentNumber")
    .agg(
        PumpSecondsOnSite_min=("PumpOnSiteSeconds", "min"),
        PumpSecondsOnSite_mean=("PumpOnSiteSeconds", "mean"),
        PumpSecondsOnSite_max=("PumpOnSiteSeconds", "max"),
        TurnoutTimeSeconds_min=("TurnoutTimeSeconds", "min"),
        TurnoutTimeSeconds_mean=("TurnoutTimeSeconds", "mean"),
        TurnoutTimeSeconds_max=("TurnoutTimeSeconds", "max"),
        TravelTimeSeconds_min=("TravelTimeSeconds", "min"),
        TravelTimeSeconds_mean=("TravelTimeSeconds", "mean"),
        TravelTimeSeconds_max=("TravelTimeSeconds", "max"),
    )    
    .reset_index()
)
display(aggregated.head())

# jointure de l'agrégat sur le dataframe pour ajouter les colonnes de temps
merge = pd.merge(
    df, aggregated, left_on="IncidentNumber", right_on="IncidentNumber", how="left"
).sort_values(by=["IncidentNumber", "PumpOrder"])

display(merge.head(10))
print(merge.shape)

In [ ]:
# Pas de traitement particulier sur PlusCode_Code, car les lignes Add et RCA ne sont pas des lignes en doublons d'un Initial, et peuvent donc être traitées comme initial

# suppression des duplicatas pour ne garder qu'une ligne par incidents
merge = merge.drop_duplicates(subset=["IncidentNumber"], keep="first")
merge.IncidentNumber.duplicated().sum()

In [71]:
# suppression des données inutiles, en doublon métier, ou non présente au moment de la prédiction
merge = merge.drop(
    [
        "CalYear",
        "HourOfCall",
        "ResourceMobilisationId",
        "Resource_Code",
        "PerformanceReporting",
        "DateAndTimeMobilised",
        "DateAndTimeMobile",
        "DateAndTimeArrived",
        "TurnoutTimeSeconds",
        "TravelTimeSeconds",
        "PumpOnSiteSeconds",
        "AttendanceTimeSeconds",
        "DateAndTimeLeft",
        "DateAndTimeReturned",
        "DeployedFromStation_Code",
        "DeployedFromStation_Name",
        "DeployedFromLocation",
        "PumpOrder",
        "PlusCode_Code",
        "PlusCode_Description",
        "DelayCodeId",
        "DelayCode_Description",
    ],
    axis=1,
)

In [72]:
# sauvegarde d'un fichier temporaire
merge.to_csv("../data/PreMobilisations.csv", sep=";", index=False)

In [ ]:
# ------------------------- Merge Incidents et Mobilisations -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Charge les dataset temporaire
df_incidents = pd.read_csv(
    "../data/PreIncidents.csv", sep=";", low_memory=False
)
df_mobilisations = pd.read_csv(
    "../data/PreMobilisations.csv", sep=";", low_memory=False
)

display(df_incidents.head())
display(df_incidents.info())

display(df_mobilisations.head())
display(df_mobilisations.info())

In [ ]:
# Merge
df = pd.merge(
    df_incidents,
    df_mobilisations,
    left_on="IncidentNumber",
    right_on="IncidentNumber",
    how="left",
)
df = df.reset_index(drop=True)

# on vérifie qu'il n'y a pas de doublon sur IncidentNumber avant de la supprimer
print("IncidentNumber na :", df.IncidentNumber.isna().sum())

display(df.head())
display(df.info())

In [75]:
# sauvegarde d'un fichier temporaire
df.to_csv("../data/PreProcessTemp.csv", sep=";", index=False)

In [ ]:
# Nettoyage du fichier PreProcessTemp. Pas de doublon sur IncidentNumber, vérifié juste avant
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/PreProcessTemp.csv", sep=";", low_memory=False)

display(df.head())
display(df.info())


In [ ]:
# Gestion des NA

def ShowNa():
    # Affichage des NA
    na_counts = df.isna().sum()
    print(na_counts[na_counts > 0],"\n")

    print("SpecialServiceType est nullable, donc normal")
    df_temp = df.drop("SpecialServiceType", axis=1)

    print("\ntotal ligne na", df_temp.isna().any(axis=1).sum())
    print("Total de lignes", len(df))
    print("Poucentage", df_temp.isna().any(axis=1).sum() / len(df) * 100, "%")

    print("\nligne par année")
    # display(df.CalYear.value_counts())

    # print("ligne na par année")
    # display(df_temp.loc[df_temp.isna().any(axis=1), "CalYear"].value_counts())
    # print("Les lignes avec des na sont principalments des lignes d'avant 2014")

    # years = []
    # for year in range(df_temp.CalYear.min(), df_temp.CalYear.max() + 1):
    #     total = len(df_temp.loc[df_temp.CalYear == year])
    #     na = df_temp[df_temp.CalYear == year].isna().any(axis=1).sum()
    #     years.append([year, total, na, na / total * 100])

    years = []
    for year in range(df_temp.CalYear.min(), df_temp.CalYear.max() + 1):
        for col in df_temp.columns:
            total = len(df_temp.loc[df_temp.CalYear == year])
            na = df_temp.loc[df_temp.CalYear == year, col].isna().sum()
            if na > 0:
                years.append([year, col, total, na, na / total * 100])

    df_years = pd.DataFrame(years, columns=["Année", "Colonne", "Nb lignes", "Na", "%Na"])
    # display(df_years)
    display(df_years.groupby(["Année", "Colonne"]).min())
    display(df_temp.loc[df_temp.USRN.isna()].head(5))


ShowNa()

# supprime les lignes avec très peu de NA
# les lignes avec des USRS vides datent d'avant 2015, on les supprime
df = df.dropna(axis = 0, how = 'any', subset =["PropertyType", "AddressQualifier", "IncGeo_WardCode", "NumPumpsAttending", "USRN"])
ShowNa()

# Il n'existe pas toujours de ligne d'intervention pour les lignes d'Incidents, on supprime les lignes sans informations de mobilisation, et parfois l'information est partiellement saisie
df = df.dropna(axis=0, how="any", subset=["PumpSecondsOnSite_min", "TurnoutTimeSeconds_min", "TravelTimeSeconds_min", "FirstPumpArriving_DeployedFromStation"])
ShowNa()


In [ ]:
df.head()

In [ ]:
# Traitmeent des catégories. Uniquement des catégories nominales
# possibilité Test Anova,


def ShowValues(col):
    print(col, len(df[col].unique()))
    # print(df[col].value_counts())

cols_categ = [
    "StopCodeDescription",
    "SpecialServiceType",
    "PropertyType",
    "AddressQualifier",
    "Postcode_district",
    "USRN",
    "IncGeo_BoroughCode",
    "IncGeo_WardCode",
    "FirstPumpArriving_DeployedFromStation",
]

# nb de valeurs par catégories
for col in cols_categ:
    ShowValues(col)

cols_cible = [
    "PumpSecondsOnSite_min",
    "PumpSecondsOnSite_mean",
    "PumpSecondsOnSite_max",
    "TurnoutTimeSeconds_min",
    "TurnoutTimeSeconds_mean",
    "TurnoutTimeSeconds_max",
    "TravelTimeSeconds_min",
    "TravelTimeSeconds_mean",
    "TravelTimeSeconds_max",
]

cols_cible_cat = ["FirstPumpArriving_DeployedFromStation", "NumPumpsAttending"]

# prépare les X et y de tests
X = df.drop(cols_cible, axis=1)
list_y = []
for col in cols_cible:
    list_y.append([col, df[col]])

df_temp = df[df.CalYear > 2020]

# Sélection de caractéristiques récursive
import statsmodels.api
from statsmodels.api import stats 
import statsmodels.formula.api as smf

cols_to_check = "+".join(cols_categ)
for col in cols_cible:
    print("Test de", col)
    result = smf.ols(f"{col} ~ {cols_to_check}", data=df_temp).fit()
    table = stats.anova_lm(result)
    display(table)
    break

# pour les temps de mobilisation on considère
model = smf.ols(
    formula="TravelTimeSeconds_min ~ StopCodeDescription + SpecialServiceType + PropertyType + USRN + HourOfCall",
    data=df_temp,
)
results = model.fit()
# Résumé des résultats
display(results.summary())

model = smf.ols(
    formula="PumpSecondsOnSite_min ~ StopCodeDescription + SpecialServiceType + PropertyType + USRN",
    data=df_temp,
)
results = model.fit()
results_df = pd.DataFrame({"coefficients": results.params, "pvalues": results.pvalues})
# Filtrer pour ne garder que les p-values <= 0.05
significant_results = results_df[results_df["pvalues"] <= 0.05]
# Afficher les résultats filtrés
display(significant_results)


# StopCodeDescription 8
# SpecialServiceType 22
# PropertyType 291
# AddressQualifier 11
# Postcode_district 322
# USRN 53339
# IncGeo_BoroughCode 33
# IncGeo_WardCode 843
# FirstPumpArriving_DeployedFromStation 117

# les cibles
# concordance acec les cibles

# PumpSecondsOnSite_min
# PumpSecondsOnSite_mean
# PumpSecondsOnSite_max
# TurnoutTimeSeconds_min
# TurnoutTimeSeconds_mean
# TurnoutTimeSeconds_max
# TravelTimeSeconds_min
# TravelTimeSeconds_mean
# TravelTimeSeconds_max
# FirstPumpArriving_DeployedFromStation
# NumPumpsAttending

In [ ]:
# supprime IncidentNumber qui n'est plus nécessaire et ne doit pas être une donnée d'entrainement
df = df.drop(["IncidentNumber"], axis=1)

In [ ]:
save